In [17]:
!pip install langchain langchain-community langchain-openai faiss-cpu pandas openpyxl

  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_openai-0.3.6-py3-none-any.whl.metadata (2.3 kB)
  Using cached faiss_cpu-1.10.0-cp310-cp310-win_amd64.whl.metadata (4.5 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-0.3.19-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.3.18-py3-none-any.whl (2.5 MB)
Using cached langchain_openai-0.3.6-py3-none-any.whl (54 kB)
Using cached faiss_cpu-1.10.0-cp310-cp310-win_amd64.whl (13.7 MB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [19]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력, OpenAI API 키를 환경 변수(os.environ)에 저장

from langchain_openai import ChatOpenAI # 랭체인을 사용하여 OpenAI의 GPT-4o 모델을 불러오는 클래스
llm = ChatOpenAI(
    model="gpt-4o",
)

In [20]:
import pandas as pd # 엑셀 데이터를 불러와서 전처리
from langchain_openai import OpenAIEmbeddings # 텍스트 데이터를 벡터(Embedding)로 변환
from langchain.vectorstores import FAISS # 벡터 데이터를 저장하고 유사도 검색을 수행
from langchain.llms import OpenAI # GPT-4o 모델을 활용하여 검색된 데이터를 바탕으로 답변 생성
from langchain.chains import RetrievalQA # 검색된 데이터를 기반으로 LLM(GPT-4)이 질의응답(QA) 수행

# 엑셀 파일(car_prices.xlsx)을 불러와서 데이터프레임(df)에 저장
file_path = "d:/data/car_prices.xlsx"
df = pd.read_excel(file_path)

# 데이터 전처리 (데이터를 벡터화하기 전에 텍스트 형태로 변환)
df["text"] = df.apply(lambda row: f"{row['제조사']} {row['모델']} 가격: {row['가격(만원)']}만원", axis=1)

# 문장을 벡터(숫자 배열)로 변환
embeddings = OpenAIEmbeddings()

# 벡터로 변환 후 FAISS 인덱스에 저장 
vectorstore = FAISS.from_texts(df["text"].tolist(), embeddings)

# 추후 빠르게 불러올 수 있도록 FAISS 인덱스를 저장
faiss_db_path = "d:/data/faiss_car_prices"
vectorstore.save_local(faiss_db_path)

# 저장된 FAISS 데이터베이스를 로드하여 재사용
vectorstore = FAISS.load_local(faiss_db_path, embeddings, allow_dangerous_deserialization=True, )

# 유사한 데이터를 검색할 수 있는 retriever 생성, 최대 5개의 유사한 결과를 반환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# 랭체인의 RetrievalQA를 사용하여 질의응답 시스템 생성
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# 사용자의 질문에 대한 답변 생성
query = "현대에서 가장 비싼 자동차는 무엇인가요?"
response = qa_chain.invoke(query)

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 현대에서 가장 비싼 자동차는 무엇인가요?
💡 답변: {'query': '현대에서 가장 비싼 자동차는 무엇인가요?', 'result': '제공된 정보에는 아우디와 볼보 자동차의 가격만 나와 있으므로, 현대 자동차에 관한 정보는 포함되어 있지 않습니다. 현대에서 가장 비싼 자동차에 대한 정보는 별도의 자료를 참고해 주시기 바랍니다.'}


In [21]:
query = "아반떼의 제조사는?"
response = qa_chain.invoke(query)

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 아반떼의 제조사는?
💡 답변: {'query': '아반떼의 제조사는?', 'result': '아반떼는 현대자동차에서 제조합니다.'}


In [22]:
query = "1억 이상하는 모든 자동차 모델을 알려줘?"
response = qa_chain.invoke(query)

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 1억 이상하는 모든 자동차 모델을 알려줘?
💡 답변: {'query': '1억 이상하는 모든 자동차 모델을 알려줘?', 'result': '제공된 정보를 보면, 1억(10,000만원) 이상인 가격을 가진 자동차 모델은 두 가지입니다.\n\n• 볼보 XC60  \n• 맥라렌 720S\n\n따라서 1억 이상하는 자동차 모델은 볼보 XC60과 맥라렌 720S입니다.'}


In [23]:
query = "현대에서 가장 비싼 가격의 모델은?"
response = qa_chain.invoke(query)

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 현대에서 가장 비싼 가격의 모델은?
💡 답변: {'query': '현대에서 가장 비싼 가격의 모델은?', 'result': '현대에서 제공된 모델 중 가장 비싼 모델은 현대 소나타로, 가격은 5,485,000만원입니다.'}
